# 2. Working with Text Data

> LLMs from Scratch

- Minjae Gwon
  <minjae.gwon@postech.ac.kr>
  <https://bxta.kr>

- ML Lab
  <https://ml.postech.ac.kr>

- CompSec Lab
  <https://compsec.postech.ac.kr>

## 2.1 Understanding Word Embeddings

### Embeddings

- 데이터를 벡터의 형태로 변환하는 것
  - 텍스트, 이미지 등의 이산 데이터를 continuous vector space에 올리는 매핑
- LLM을 포함한 deep neural network들은 텍스트를 바로 처리할 수 없기 때문에 필요

### Word Embeddings

- Example: *Word2Vec*
  - 가정: 비슷한 의미를 가진 단어들은 비슷한 벡터로 매핑됨
  - 2차원 공간에 word embedding을 시각화하면 비슷한 의미를 가진 단어들이 서로 가까이 위치함
    - ![](https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp)
- Real World: LLMs
  - 자기 자신만의 embedding layer를 input layers 중 하나로 구성하고 학습 단계에 업데이트 하는 것이 일반적
    - 즉, pretrained *Word2Vec* 모델을 이용하지 않음
  - 특정 task와 data에 optimized 된 embedding을 얻을 수 있음
  - 2차원보다 훨씬 큰 차원의 embedding을 이용함

## 2.2 Tokenizing Text

### Goal

- 20,479개의 글자로 이루어진 짧은 이야기 "The Verdict"를 tokenize 하는 것

#### Exercise: Load Document

- "The Verdict"를 `document` 변수에 저장하기

In [1]:
from pathlib import Path

from llm_from_scratch.chapter_02.tests import test_document_loaded_correctly

path_of_document = Path("the-verdict.txt")

document: str = ...

test_document_loaded_correctly(document)

In [2]:
print("Total number of character:", len(document))
print(document[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


### Simple Approach: Word-Based Tokenizer

In [3]:
import re

text = "Hello, world. This, is a test."
result = re.split(r"(\s)", text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 주어진 텍스트를 토큰의 리스트로 변환하기 위해 `re.split`을 사용할 수 있음
  - `re.split`은 정규표현식을 이용해 텍스트를 나누는 함수

In [4]:
text = "Hello, world. Is this-- a test?"

splitted = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in splitted if item.strip()]

print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 문장 부호들과 같은 특수 문자들을 분리함
- 예시의 간결함을 유지하기 위해 whitespace를 삭제
  - 경우에 따라 whitespace를 유지하는 것이 이로울 수도 있음
    - e.g. Python Code (indentation 때문에)

#### Exercise: Tokenize Document

- `re.split`을 이용해 위와 같은 규칙으로 `document`를 tokenize해보자.

In [5]:
from llm_from_scratch.chapter_02.tests import test_document_tokenized_correctly

tokenized_document: list[str] = ...

test_document_tokenized_correctly(tokenized_document)

In [6]:
print("Length of `preprocessed_the_verdict`:", len(tokenized_document))
print("Samples: ", tokenized_document[:30])

Length of `preprocessed_the_verdict`: 4690
Samples:  ['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


## 2.3 Converting Tokens into Token IDs

### Goal

- 각 토큰을 고유한 정수의 ID로 변환하기
  - Embedding vector로 변환하기 위해 필요

### Vocabulary

- 각 토큰과 특수 문자에 대한 고유한 ID의 mapping
  - ![](https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp)

#### Exercise: Create Vocabulary

- 앞서 생성한 `tokenized_document`를 이용해 vocabulary를 생성해보자.
- `vocabulary`는 토큰에 대해 고유한 숫자를 매핑하는 Python dictionary여야 함
- Alphabetical order로 정렬하되 중복을 제거해야 함

In [7]:
from llm_from_scratch.chapter_02.tests import test_vocabulary_created_correctly


vocabulary: dict[str, int] = ...

test_vocabulary_created_correctly(vocabulary)

In [8]:
print("Length of `vocabulary`:", len(vocabulary))
print("Samples: ", list(vocabulary.items())[:10])

Length of `vocabulary`: 1130
Samples:  [('!', 0), ('"', 1), ("'", 2), ('(', 3), (')', 4), (',', 5), ('--', 6), ('.', 7), (':', 8), (';', 9)]


#### Exercise: Convert Text to IDs

- `vocabulary`를 이용해서 `text`를 token IDs로 변환하는 함수 `encode`를 작성해보자.

In [9]:
from llm_from_scratch.chapter_02.tests import test_encode_implemented_correctly


def encode(text: str, vocabulary: dict[str, int]) -> list[int]: ...


test_encode_implemented_correctly(encode)

In [10]:
encoded = encode("I HAD always thought Jack Gisburn", vocabulary)
print(encoded)

[53, 44, 149, 1003, 57, 38]


#### Exercise: Convert Token IDs to Text

- `vocabulary`를 이용해서 token IDs를 `text`로 변환하는 함수 `decode`를 작성해보자.

In [11]:
from llm_from_scratch.chapter_02.tests import test_decode_implemented_correctly


def decode(token_ids: list[int], vocabulary: dict[str, int]) -> str: ...


test_decode_implemented_correctly(decode)

In [12]:
decoded = decode(encoded, vocabulary)
print(decoded)

I HAD always thought Jack Gisburn


#### Exercise: Implement Simple Tokenizer

- 위에서 작성한 `encode`와 `decode`를 이용해 `SimpleTokenizerV1` 클래스를 작성해보자.
- `encode` 메소드는 `text`를 token IDs로 변환하고, `decode` 메소드는 token IDs를 텍스트로 변환해야 함

In [13]:
from llm_from_scratch.chapter_02.protocols import TokenizerProtocol
from llm_from_scratch.chapter_02.tests import (
    test_simple_tokenizer_v1_implemented_correctly,
)


class SimpleTokenizerV1(TokenizerProtocol):
    def __init__(self, vocabulary: dict[str, int]) -> None: ...

    def encode(self, text: str) -> list[int]: ...

    def decode(self, token_ids: list[int]) -> str: ...


test_simple_tokenizer_v1_implemented_correctly(SimpleTokenizerV1)

In [14]:
tokenizer = SimpleTokenizerV1(vocabulary)

encoded = tokenizer.encode("I HAD always thought Jack Gisburn")
decoded = tokenizer.decode(encoded)

print(encoded, decoded)

[53, 44, 149, 1003, 57, 38] I HAD always thought Jack Gisburn


## 2.4 Adding Special Context Tokens

### Goals

In [15]:
tokenizer = SimpleTokenizerV1(vocabulary)

try:
    tokenizer.encode("Hello, do you like tea?")
except KeyError as e:
    print("The token is not in the vocabulary")

The token is not in the vocabulary


- `SimpleTokenizerV1`은 vocabulary에 없는 단어를 처리할 수 없음
- 이를 해결하기 위해 special context tokens을 이용할 것임
- 또한 special context token을 통해 모델이 문맥이나 다른 정보들을 이해할 수 있도록 하는 방법을 알아볼 것임
  - e.g. 문장의 시작과 끝을 알리는 token

#### Exercise: Extend Vocabulary

- 기존 `vocabulary`에 `<|endoftext|>`, `<|unk|>` token을 추가해보자.
  - `<|endoftext|>`: 문장의 끝을 알리는 token
  - `<|unk|>`: vocabulary에 없는 단어를 나타내는 token

In [16]:
from llm_from_scratch.chapter_02.tests import test_extended_vocabulary_created_correctly

vocabulary: dict[str, int] = ...

test_extended_vocabulary_created_correctly(vocabulary)

In [17]:
print(list(vocabulary.items())[-3:])

[('yourself', 1129), ('<|endoftext|>', 1130), ('<|unk|>', 1131)]


#### Exercise: Implement Simple Tokenizer with Special Tokens

- `SimpleTokenizerV2` 클래스를 작성해보자
- `SimpleTokenizerV1`와 같은 형태를 따름
- 다만 `SimpleTokenizerV2`는 `SimpleTokenizerV1`와 달리 vocabulary에 없는 토큰을 처리할 수 있어야 함

In [18]:
from llm_from_scratch.chapter_02.tests import (
    test_simple_tokenizer_v2_implemented_correctly,
)


class SimpleTokenizerV2:
    def __init__(self, vocabulary: dict[str, int]) -> None: ...

    def encode(self, text: str) -> list[int]: ...

    def decode(self, token_ids: list[int]) -> str: ...


test_simple_tokenizer_v2_implemented_correctly(SimpleTokenizerV2)

In [19]:
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace."

tokenizer = SimpleTokenizerV2(vocabulary)

encoded = tokenizer.encode(text)
print(encoded)

decoded = tokenizer.decode(encoded)
print(decoded)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
<|unk|> , do you like tea ? <|endoftext|> In the sunlit terraces of the <|unk|> .


### Special Tokens in Real World

- `BOS` (Beginning of Sequence)
  - LLM에게 컨텐츠의 시작을 알리는 역할을 함
- `EOS` (End of Sequence)
  - LLM에게 컨텐츠의 끝을 알리는 역할을 함
- `PAD` (Padding)
  - LLM에게 sequence의 길이를 맞추기 위한 padding을 하는 역할을 함
  - e.g. Batch 내에서 길이가 다른 sequence들을 padding하여 같은 길이로 만들어줌

## 2.5 Byte Pair Encoding

### Example: `tiktoken`

- `tiktoken`은 OpenAI에서 maintain하는 BPE tokenizer 라이브러리

In [20]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
)

encoded = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(encoded)

decoded = tokenizer.decode(encoded)

print(decoded)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]
Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.


### Observations

- `<|endoftext|>`의 경우 상대적으로 큰 ID를 부여 받음
  - GPT-2, GPT-3 등에서 쓰이는 이 tokenizer의 vocabulary 크기가 상대적으로 큰 50,257임을 알 수 있음.
- "someunknownPlace"와 같이 vocabulary에 없는 단어도 처리할 수 있음
  - `<|unk|>` token을 이용하지 않아도 됨

### Algorithm (Brief)

![](https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp)

- OOV 토큰을 더 작은 subword나 character로 나누어서 모르는 토큰에 대해서도 핸들 가능하게 함

## 2.6 Data Sampling with a Sliding Window

### Goal

![](https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp)

- Input-Target 쌍을 sliding window의 개념을 통해 생성할 것임
  - Embedding을 만들기 위해 필요함

### Input-Target Pairs

- 위 figure에 묘사된 next-word prediction task는 아래 코드와 같이 표현될 수 있음

In [21]:
sample_of_encoded: list[int] = tokenizer.encode(document)[50:]

for i in range(1, 5):
    context = sample_of_encoded[:i]
    target_id = sample_of_encoded[i]
    print(context, "---->", target_id)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


- 더 알아보기 쉽게 텍스트로 표현하면 아래와 같음

In [22]:
for i in range(1, 5):
    context = sample_of_encoded[:i]
    target_id = sample_of_encoded[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([target_id]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- (Personal Opinion -- Why there is no reason in the book? idk)
  - 하지만 보통 input chunk와 target chunk의 사이즈를 같게 만듬
    - Seq2Seq 모델의 경우 필수적임
    - 같은 사이즈의 배치로 올려야 GPU 메모리를 더 효율적으로 이용할 수 있음
    - 입력과 출력의 청크 사이즈가 같은 것이 구현이 쉬움
- Input chunk와 target chunk의 사이즈가 같도록, input chunk를 window로 생각하고 target chunk를 input chunk로부터 1칸을 민 chunk로 고려할 것임

![](https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123)

#### Exercise: Single Pair of Input Chunk and Target Chunk

- token IDs가 주어졌을 때 `start_index`로부터 `context_size` 크기의 window `input_chunk`, `target_chunk`를 반환하는 함수 `input_chunk_and_target_chunk`을 구현해보자.
  - `input_chunk`에는 input token들이 할당됨
  - `target_chunk`에는 target token들이 포함된 token들이 할당됨
    - input으로부터 1칸 shift 된 결과물

In [23]:
from llm_from_scratch.chapter_02.tests import (
    test_input_chunk_and_target_chunk_created_correctly,
)


def input_chunk_and_target_chunk(
    token_ids: list[int], start_index: int, context_size: int
) -> tuple[list[int], list[int]]:
    return input_chunk(token_ids, start_index, context_size), target_chunk(
        token_ids, start_index, context_size
    )


def input_chunk(
    token_ids: list[int], start_index: int, context_size: int
) -> list[int]: ...


def target_chunk(
    token_ids: list[int], start_index: int, context_size: int
) -> list[int]: ...


test_input_chunk_and_target_chunk_created_correctly(input_chunk_and_target_chunk)

In [24]:
partial_encoded: list[int] = tokenizer.encode(document)[50:]
context_size = 4

_input_chunk, _target_chunk = input_chunk_and_target_chunk(
    partial_encoded, 0, context_size
)

print(f" input_chunk: {_input_chunk}")
print(f"target_chunk:      {_target_chunk}")

 input_chunk: [290, 4920, 2241, 287]
target_chunk:      [4920, 2241, 287, 257]


#### Exercise: Dataset

- 텍스트와 tokenizer가 주어졌을 때 input chunk와 target chunk를 반환할 수 있는 torch Dataset `GPTDatasetV1`을 구현해보자.

In [25]:
from typing import Callable
import torch
from torch.utils.data import Dataset
from torch import Tensor

from llm_from_scratch.chapter_02.tests import test_gpt_dataset_v1


class GPTDatasetV1(Dataset[tuple[Tensor, Tensor]]):
    def __init__(
        self,
        text: str,
        encode: Callable[[str], list[int]],
        context_size: int,
        stride: int,
    ) -> None:
        super().__init__()
        ...

    def __len__(self): ...

    def __getitem__(self, index: int | slice): ...


test_gpt_dataset_v1(GPTDatasetV1)

- 아래 코드를 실행한 결과를 확인하여 위의 데이터셋이 잘 구성되었는지 확인할 수 있다.
  - `Dataloader`를 통해서 데이터를 불러오는 코드이다.

In [26]:
from torch.utils.data import DataLoader


def create_dataloader_v1(
    text: str,
    batch_size: int = 4,
    context_size: int = 256,
    stride: int = 128,
    shuffle: bool = True,
    drop_last: bool = True,
):
    tokenizer = tiktoken.get_encoding("gpt2")
    encode = tokenizer.encode

    dataset = GPTDatasetV1(text, encode, context_size, stride)

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
    )

    return dataloader


dataloader = create_dataloader_v1(
    document, batch_size=1, context_size=4, stride=1, shuffle=False
)

data_iterator = iter(dataloader)

first_batch = next(data_iterator)

print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


- `first_batch`는 두 개의 텐서로 이루어져 있음
  - 첫 번째 텐서는 input chunks, 두 번째 텐서는 target chunks를 나타냄

In [27]:
second_batch = next(data_iterator)

print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- `second_batch`를 통해 stride의 역할을 확인할 수 있음
  - 위에서 확인한 `first_batch`와 비교하여 한 칸이 밀려있는지 확인해보자.
  - `context_size`와 동일하게 `stride`를 설정할 경우 배치들 간에 input이 overlap 되는 상황을 방지할 수 있음

![](https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp)

## 2.7 Creating Token Embeddings

### Embedding

- Token IDs를 continuous vector space로 변환하는 것
  - 각 토큰을 고유한 벡터로 매핑하는 것
  - LLMs들이 back-propagation을 통해 학습할 수 있도록 함

#### Exercise: Torch Embedding

- PyTorch의 `nn.Embedding`을 이용해서 embedding layer를 생성해보자.
- Vocabulary의 크기는 6, token embedding의 차원은 3으로 생각

In [28]:
from llm_from_scratch.chapter_02.tests import test_simple_embedding


torch.manual_seed(42)
embedding: torch.nn.Embedding = ...

test_simple_embedding(embedding)

In [29]:
print(embedding.weight)

Parameter containing:
tensor([[ 1.9269,  1.4873, -0.4974],
        [ 0.4396, -0.7581,  1.0783],
        [ 0.8008,  1.6806,  0.3559],
        [-0.6866,  0.6105,  1.3347],
        [-0.2316,  0.0418, -0.2516],
        [ 0.8599, -0.3097, -0.3957]], requires_grad=True)


In [30]:
print(embedding(torch.tensor([3])))

tensor([[-0.6866,  0.6105,  1.3347]], grad_fn=<EmbeddingBackward0>)


- 위에서 생성한 embedding을 이용해서 token ID `3`에 상응하는 embedding vector를 구하면 위 embedding에서 4번째 row를 가져오는 것을 확인할 수 있다.
  - 다시 말해서, embedding layer는 단순 lookup table로 이해할 수 있음

## 2.8 Encoding Word Positions

### Positional Embeddings

![](https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp)

- 토큰의 위치 정보를 추가로 인코딩 하기 위해 "Positional Embedding"을 이용함
  - "Token embedding"만 이용할 경우 토큰의 위치에 상관 없이 항상 같은 embedding이 할당됨
    - 따라서 토큰의 위치 정보를 추가로 인코딩해야 함
  - LLM이 문맥이나 토큰 사이의 관계를 더 잘 이해할 수 있도록 돕는 역할을 수행함
- Types
  - Absolute Positional Embeddings
    - Input sequence의 각 위치에 대해 고유한 embedding을 더하는 방법
    - ![](https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp)
  - Relative Positional Embeddings
    - 토큰 사이의 상대적인 위치나 거리를 인코딩하는 방법
    - 다양한 길이의 sequence에 대해 일반화된 embedding을 제공할 수 있음


#### Exercise: Positional Embeddings

In [31]:
dataloader = create_dataloader_v1(
    document, batch_size=8, context_size=4, stride=4, shuffle=False
)

data_iterator = iter(dataloader)

first_batch = next(data_iterator)

inputs, targets = first_batch

- 우선 출력이 256차원이 되도록 embedding을 생성해보자.
- Hint: `tokenizer`의 vocabulary size는 50257임

In [32]:
from llm_from_scratch.chapter_02.tests import test_token_embedding


torch.manual_seed(42)

embedding: torch.nn.Embedding = ...

test_token_embedding(embedding)

- `inputs`의 embedding을 구하자.

In [33]:
from llm_from_scratch.chapter_02.tests import test_token_embeddings


token_embeddings: torch.Tensor = ...

test_token_embeddings(token_embeddings)

In [34]:
print(token_embeddings.shape)

torch.Size([8, 4, 256])


- Absolute position embeddings를 생성하자.
  - Hint: Embedding layer의 row가 몇 개가 되어야 할까?
  - Hint: Embedding layer의 출력 차원은 몇 차원이 되어야 할까?
  - Hint: torch.arange

In [35]:
from llm_from_scratch.chapter_02.tests import (
    test_position_embedding,
    test_position_embeddings,
)


torch.manual_seed(42)

position_embedding: torch.nn.Embedding = ...

test_position_embedding(position_embedding)

position_embeddings: torch.Tensor = ...

test_position_embeddings(position_embeddings)

In [36]:
print(position_embeddings)

tensor([[ 1.9269,  1.4873,  0.9007,  ...,  0.5655,  0.5058,  0.2225],
        [-0.6855,  0.5636, -1.5072,  ...,  0.4232, -0.3389,  0.5180],
        [-1.3638,  0.1930, -0.6103,  ..., -1.6034, -0.4298,  0.5762],
        [ 0.3444, -3.1016, -1.4587,  ...,  1.1085,  0.5544,  1.5818]],
       grad_fn=<EmbeddingBackward0>)


- Input embeddings를 구해보자.
  - Token embedding과 positional embedding을 결합하자.

In [37]:
from llm_from_scratch.chapter_02.tests import test_input_embeddings


input_embeddings: torch.Tensor = ...

test_input_embeddings(input_embeddings)

In [38]:
print(input_embeddings)

tensor([[[ 1.4662e+00,  2.3496e+00,  8.7182e-01,  ..., -1.6724e-01,
           2.8568e+00, -1.6840e-01],
         [-1.2675e+00,  6.0994e-01, -1.7767e+00,  ..., -6.1660e-02,
           6.2235e-01,  1.5896e+00],
         [-1.9898e+00,  3.7392e-01, -6.4176e-01,  ..., -5.7213e-02,
          -6.2085e-01, -1.3078e+00],
         [ 2.0358e-01, -1.9663e+00, -1.2676e+00,  ..., -9.3234e-03,
           1.4072e+00,  1.9885e+00]],

        [[ 2.1036e+00,  1.6725e+00, -1.1920e+00,  ...,  2.4488e+00,
           1.6690e+00,  3.6603e-01],
         [-4.1414e-01,  9.4160e-01, -1.2794e+00,  ...,  2.0839e-01,
          -1.7464e+00, -5.3957e-01],
         [-5.2307e-01,  3.5381e-01,  1.4262e+00,  ..., -4.7622e-01,
           3.5551e-01, -3.0002e-01],
         [-6.8878e-01, -4.9592e+00, -1.2256e+00,  ...,  4.1810e-01,
           3.7742e-01,  1.1978e+00]],

        [[ 7.8653e-01,  3.2022e+00,  1.2175e+00,  ...,  4.2110e-01,
           1.6269e+00,  1.5479e+00],
         [ 5.7563e-01,  1.2872e+00, -2.7510e+00,  .